In [1]:
import numpy as np

In [2]:
# Client Definition
class Client:
  def __init__(self, client_id, data, labels, lr = 0.01):
    self.client_id = client_id
    self.data = data
    self.labels = labels
    self.lr = lr
    self.weights = np.random.rand(data.shape[1]) # simple linear model

  def train_local(self, epochs = 5):
    """"Local Training using simple gradient descent for linear regression"""
    for _ in range(epochs):
      predictions = self.data.dot(self.weights)
      errors = predictions - self.labels
      gradient = self.data.T.dot(errors) / len(self.data)
      self.weights -= self.lr * gradient

    print(f"Client {self.client_id} finished local training")
    return self.weights


In [3]:
# Server Definition
class Server:
  def __init__(self, num_clients):
    self.num_clients = num_clients
    self.global_weights = None

  def aggregator(self, client_weights):
    """Federated averaging of all client model parameters"""
    self.global_weights = np.mean(client_weights, axis = 0)
    print("\nServer aggregated weights using Federated Averaging..\n")
    return self.global_weights

  def send_weights(self):
    return self.global_weights

In [9]:
# SIMULATION SETUP
def generate_client_data(num_clients=3, samples_per_client=30):
    """
    Generate synthetic data for each client.
    y = 2x + noise
    """
    clients = []
    for i in range(num_clients):
        x = np.random.rand(samples_per_client, 1)
        y = 2 * x.squeeze() + np.random.randn(samples_per_client) * 0.1
        clients.append((x, y))
    return clients


def federated_learning_simulation(num_clients=3, rounds=5):
    # Create server
    server = Server(num_clients)

    # Generate client data
    client_data = generate_client_data(num_clients)

    # Initialize clients
    clients = [
        Client(i, data=x, labels=y)
        for i, (x, y) in enumerate(client_data)
    ]

    # GLOBAL MODEL INITIALIZATION
    server.global_weights = np.zeros(clients[0].weights.shape)

    for r in range(rounds):
        print(f"\n================ Federated Learning Round {r+1} ================\n")

        client_weights = []

        # SERVER sends global weights to all clients
        for client in clients:
            client.weights = server.send_weights().copy()

        # CLIENTS train on local data
        for client in clients:
            w = client.train_local()
            client_weights.append(w)

        # SERVER aggregates client updates
        global_w = server.aggregate(client_weights)

        print(f"Updated Global Weights: {global_w}\n")

    print("\nTraining completed!")
    print(f"Final Global Model Weights: {server.global_weights}")


if __name__ == "__main__":
    federated_learning_simulation()



================ Federated Learning Round 1 ================

Client 0 finished local training.
Client 1 finished local training.
Client 2 finished local training.

Server aggregated weights using Federated Averaging...

Updated Global Weights: [0.02908466]


================ Federated Learning Round 2 ================

Client 0 finished local training.
Client 1 finished local training.
Client 2 finished local training.

Server aggregated weights using Federated Averaging...

Updated Global Weights: [0.05774321]


================ Federated Learning Round 3 ================

Client 0 finished local training.
Client 1 finished local training.
Client 2 finished local training.

Server aggregated weights using Federated Averaging...

Updated Global Weights: [0.08598189]


================ Federated Learning Round 4 ================

Client 0 finished local training.
Client 1 finished local training.
Client 2 finished local training.

Server aggregated weights using Federated Averaging...